In [3]:
#%pip install torch_geometric

In [4]:
import logging

from collections import namedtuple


import numpy as np
import pandas as pd

import model as model
import os
import torch
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
import graph

In [5]:
from graph import load_graphs, SparseGraph, graph_from_sparse,construct_graphs,save_graphs,graph_from_files,save_graph_combined,load_graph_combined

In [6]:
import os
folder_path = "/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/raw/pi-_10GeV_4ly_1e-4_inclined_4ly/"
lista=os.listdir(folder_path)

In [7]:
file_node_df = [os.path.join(folder_path, file) for file in lista if "node_df" in file and "viewx" in file]
file_edge_df = [os.path.join(folder_path, file) for file in lista if "edge_df" in file and "viewx" in file]

In [8]:
def extract_number_from_filename(filename):
    start_index = filename.find("-evt-") + len("-evt-")
    end_index = filename.find("_", start_index)
    return int(filename[start_index:end_index])

# Ordina i file basati sul numero dopo "-evt-" nel loro nome
file_node_df_sorted = sorted(file_node_df, key=extract_number_from_filename)
file_edge_df_sorted = sorted(file_edge_df, key=extract_number_from_filename)


In [26]:
# node_df_pi=pd.read_parquet('/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/raw/pi-_10GeV_4ly_1e-4_inclined_4ly/TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-0_pitch0.25node_df.parquet')
# edge_df_pi=pd.read_parquet('/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/raw/pi-_10GeV_4ly_1e-4_inclined_4ly/TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-0_pitch0.25edge_df.parquet')

In [15]:
edge_df_pi

,idx_node_s,x_hit_s,zx_hit_s,dx_hit_s,xMC_s,ly_s,PE_s,PID_s,idx_node_t,x_hit_t,...,dx_hit_t,xMC_t,ly_t,PE_t,PID_t,Ev,angx_row,angx_MC,deangx,edge_class
0,0,-10.735,-30.466,1.21,-10.722050,0,19,-211.0,1,-13.543,...,1.74,-13.528648,1,26,-211.0,6,270.134628,270.134561,-0.101611,1
1,0,-10.735,-30.466,1.21,-10.722050,0,19,-211.0,2,14.266,...,2.68,-13.528648,1,7,-999.0,6,268.801515,270.134561,-0.140919,0
2,1,-13.543,1164.578,1.74,-13.528648,1,26,-211.0,3,-13.591,...,1.53,-13.616448,2,24,-211.0,6,270.073564,270.134561,-3.551003,1
3,2,14.266,1164.578,2.68,-13.528648,1,7,-999.0,3,-13.591,...,1.53,-13.616448,2,24,-211.0,6,306.691211,270.134561,-3.041053,0
4,3,-13.591,1201.963,1.53,-13.616448,2,24,-211.0,4,-15.444,...,1.85,-14.991177,3,17,-211.0,6,270.181374,270.134561,-0.234983,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50064,3,9.567,1201.928,1.31,9.636515,2,27,-211.0,4,10.552,...,1.39,10.599679,3,14,-211.0,9998,269.903592,269.905730,-0.186945,1
50065,0,2.303,-30.575,0.33,2.502693,0,26,-211.0,1,10.615,...,1.22,10.606787,1,28,-211.0,9999,269.601490,269.611457,-0.060592,1
50066,1,10.615,1164.463,1.22,10.606787,1,28,-211.0,2,10.804,...,1.53,10.861580,2,12,-211.0,9999,269.711785,269.611457,-2.984056,1
50067,2,10.804,1202.035,1.53,10.861580,2,12,-211.0,3,12.816,...,2.18,14.832120,3,8,-999.0,9999,269.803111,269.611457,-0.260624,0


In [4]:
# folder_name='/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/raw/pi-_10GeV_4ly_1e-4_inclined_4ly/'
# file1='TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-1_pitch0.25node_df.parquet'
# file2='TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-2_pitch0.25node_df.parquet'
# fileE1='TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-1_pitch0.25edge_df.parquet'
# fileE2='TrkData_viewx_cl_TB_2023_T10_Sept_pi-Mono_10000_inclined_10000-evt-2_pitch0.25edge_df.parquet'
# lista_nodi=[folder_name+file1,folder_name+file2]
# lista_edges=[folder_name+fileE1,folder_name+fileE2]

In [16]:
# graph_sparse=construct_graphs(node_df_pi,edge_df_pi) #single file

In [7]:
edge_df=pd.read_parquet(file_edge_df_sorted[0])

In [34]:
len(file_edge_df_sorted)

991

In [25]:
graph_sparse_list=[]
for i in range(200,300):#(len(file_node_df_sorted)):
    # print(i)
    node_df=pd.read_parquet(file_node_df_sorted[i])
    edge_df=pd.read_parquet(file_edge_df_sorted[i])
    # print(edge_df.head())
    graph_sparse_list.append(construct_graphs(node_df,edge_df))

In [26]:
len(graph_sparse_list)

100

In [15]:
# graph_sparse_list

In [24]:
#single file
# graph=[]
# for i in range(0,len(graph_sparse)):
#     gr=graph_from_sparse(graph_sparse[i])
#     graph.append(gr)

In [16]:
graph=[]
for j in range(0,100):#len(graph_sparse_list)):
    graph.append([])
    for k in range(0,len(graph_sparse_list[j])):
        gr=graph_from_sparse(graph_sparse_list[j][k])
        graph[j].append(gr)

In [23]:
len(graph)

100

In [7]:
# filenames = []
# for i in range(0,len(graph)):
#     filenames.append(f'/lustrehome/federicacuna/TB_Sept_2023_ml/Data/npz_file_IN/npz_file/graph_{i}.npz')
# save_graphs(graph, filenames)

In [15]:
# data=np.load('/lustrehome/federicacuna/TB_Sept_2023_ml/Data/npz_file_IN/npz_file/graph_0.npz')
# data.files
# data['y']

In [ ]:
#####tentativo per salvare i grafi in un unico file npz

In [24]:
for i in range (100):#range(len(graph)):
    name=i+100
    save_graph_combined(f"/lustrehome/federicacuna/TB_Sept_2023_ml/Data/npz_file_IN/viewx_data_npz{name}.npz",graph[i])

IndexError: list index out of range

In [14]:
# data_combined=load_graph_combined('test_npz0.npz')

In [17]:
# data_for_train=[]
# for i in range(0,2):
#     data_for_train.extend(load_graph_combined(f'test_npz{i}.npz'))

In [ ]:
######creazione di tanti fil npz quanti sono gli eventi nei vari file salvati!!!

In [19]:
# sparse_gr_mult=graph_from_files(lista_nodi,lista_edges)